In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os

# The reason from and to gtfs generation dates are included is to ensure that different
# timeframes' gtfs service_id's trip_ids are maintained

In [2]:
# Retrieve trips.txt files for each gtfs generation dates
extract_path = "./files/extracted/"
trips_path = "/trips.txt"
gtfs_generation_dates = [item for item in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, item))]

In [3]:
def process_trips_dates_file(current_gtfs_date, next_gtfs_date):
    # Read the trips.txt file into a DataFrame
    transfers = pd.read_csv(extract_path + current_gtfs_date + trips_path)
    transfers['current_gtfs_date'] = current_gtfs_date
    transfers['next_gtfs_date'] = next_gtfs_date
    # transfers['next_gtfs_date'] = 99991231 if next_gtfs_date == None else next_gtfs_date

    return(transfers)

In [ ]:
# total_trips = pd.DataFrame()
# # Set the path to the GTFS files
# for gtfs_date in gtfs_generation_dates:
#     # calendar_file = folder_path + filename # example: calendar_20181221.txt'
#     total_trips = pd.concat([total_trips, process_trips_dates_file(gtfs_date)])

# # Group by trip_id and service_id and get the minimum value_amount in each group
# grouped_df = total_trips.groupby(["trip_id", "service_id"]).agg({"gtfs_generation_date": "min"}).reset_index()

In [20]:
total_trips = pd.DataFrame()
for i, item in enumerate(gtfs_generation_dates):
    # Set the last date for 'next_item' to be highdate
    next_item = gtfs_generation_dates[i+1] if i < len(gtfs_generation_dates)-1 else 99991231
    total_trips = pd.concat([total_trips, process_trips_dates_file(item, next_item)])

In [23]:
# Clean total_transfers down to columns that are applicable for the logic going forward
total_transfers = total_trips[['service_id','trip_id','current_gtfs_date','next_gtfs_date']]
unique_transfers = total_transfers.drop_duplicates()

In [27]:
# Generate a mappings folder if it doesn't exist
if not os.path.exists('mappings'):
    os.makedirs('mappings')

# Generate a mapping file for future reference
unique_transfers.to_csv('mappings/services_to_trips.csv', index=False)

,service_id,trip_id,current_gtfs_date,next_gtfs_date
0,A20130803WKD,A20130803WKD_000800_1..S03R,20131030,20140204
1,A20130803WKD,A20130803WKD_007300_1..N03R,20131030,20140204
2,A20130803WKD,A20130803WKD_002700_1..S03R,20131030,20140204
3,A20130803WKD,A20130803WKD_009300_1..N03R,20131030,20140204
4,A20130803WKD,A20130803WKD_004700_1..S03R,20131030,20140204
...,...,...,...,...
19649,SIR-FA2017-SI017-Sunday-00,SIR-FA2017-SI017-Sunday-00_138600_SI..S03R,20221129,99991231
19650,SIR-FA2017-SI017-Sunday-00,SIR-FA2017-SI017-Sunday-00_141100_SI..N03R,20221129,99991231
19651,SIR-FA2017-SI017-Sunday-00,SIR-FA2017-SI017-Sunday-00_141600_SI..S03R,20221129,99991231
19652,SIR-FA2017-SI017-Sunday-00,SIR-FA2017-SI017-Sunday-00_144100_SI..N03R,20221129,99991231
